In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten, Dropout,LSTM

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Raw_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/only data/raw_data.npz'

Theta_data = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/time frequency domain/RAW/Theta_merged_features.npz'



# RAW = np.load(Raw_data, allow_pickle=True)  # Allow loading pickled object arrays
# X = RAW['X'].astype('float64')
# Y = RAW['Y'].astype('float64')
# group = RAW['Group'].astype('float64')

This is a good performing model

# CNN-LSTM

In [14]:
%%capture
# Upgrade TensorFlow and TensorFlow Addons to the latest versions
!pip install --upgrade tensorflow tensorflow-addons

In [ ]:
!pip install tensorflow-addons==0.11.2
!pip install tensorflow==2.2.0

import tensorflow as tf
import tensorflow_addons as tfa

print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow Addons version: {tfa.__version__}")

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            # opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
            opt = tfa.optimizers.SWA(opt)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
            # client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Theta/CNN_LSTM/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Theta/CNN_LSTM/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


ImportError: cannot import name 'Tensor' from 'tensorflow.python.framework.ops' (/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/ops.py)

In [ ]:
global_model, metrics_df = federated_learning(Theta_data)


Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 37, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 37, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 19, 256)           0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 26s 154ms/step - loss: 1.7035 - accuracy: 0.5151 - val_loss: 1.6983 - val_accuracy: 0.5226
Epoch 2/100
29/29 [==============================] - 1s 23ms/step - loss: 1.6930 - accuracy: 0.5547 - val_loss: 1.6881 - val_accuracy: 0.5280
Epoch 3/100
29/29 [==============================] - 1s 25ms/step - loss: 1.6824 - accuracy: 0.5679 - val_loss: 1.6780 - val_accuracy: 0.5291
Epoch 4/100
29/29 [==============================] - 1s 38ms/step - loss: 1.6718 - accuracy: 0.5539 - val_loss: 1.6679 - val_accuracy: 0.5496
Epoch 5/100
29/29 [==============================] - 1s 25ms/step - loss: 1.6610 - accuracy: 0.5490 - val_loss: 1.6580 - val_accuracy: 0.5571
Epoch 6/100
29/29 [==============================] - 1s 26ms/step - loss: 1.6498 - accuracy: 0.5501 - val_loss: 1.6481 - val_accuracy: 0.5657
Epoch 7/100
29/29 [==============================] - 1s 25ms/step - loss: 1.6385 - accuracy: 0.5523 - val_loss: 1.6382 - val_accuracy: 0.5668
Epoch 8/100
29/2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 8s 81ms/step - loss: 1.7038 - accuracy: 0.5379 - val_loss: 1.6987 - val_accuracy: 0.5147
Epoch 2/100
28/28 [==============================] - 1s 23ms/step - loss: 1.6936 - accuracy: 0.5566 - val_loss: 1.6888 - val_accuracy: 0.5192
Epoch 3/100
28/28 [==============================] - 1s 27ms/step - loss: 1.6835 - accuracy: 0.5470 - val_loss: 1.6790 - val_accuracy: 0.5215
Epoch 4/100
28/28 [==============================] - 1s 33ms/step - loss: 1.6732 - accuracy: 0.5637 - val_loss: 1.6694 - val_accuracy: 0.5226
Epoch 5/100
28/28 [==============================] - 1s 31ms/step - loss: 1.6629 - accuracy: 0.5606 - val_loss: 1.6597 - val_accuracy: 0.5351
Epoch 6/100
28/28 [==============================] - 1s 31ms/step - loss: 1.6524 - accuracy: 0.5603 - val_loss: 1.6502 - val_accuracy: 0.5407
Epoch 7/100
28/28 [==============================] - 1s 34ms/step - loss: 1.6416 - accuracy: 0.5481 - val_loss: 1.6406 - val_accuracy: 0.5498
Epoch 8/100
28/28 

...

29/29 [==============================] - 8s 56ms/step - loss: 0.4795 - accuracy: 0.8456 - val_loss: 0.9814 - val_accuracy: 0.5162
Epoch 2/100
29/29 [==============================] - 1s 23ms/step - loss: 0.4795 - accuracy: 0.8473 - val_loss: 0.9086 - val_accuracy: 0.5172
Epoch 3/100
29/29 [==============================] - 1s 24ms/step - loss: 0.4648 - accuracy: 0.8583 - val_loss: 0.8914 - val_accuracy: 0.5183
Epoch 4/100
29/29 [==============================] - 1s 22ms/step - loss: 0.4605 - accuracy: 0.8578 - val_loss: 0.8113 - val_accuracy: 0.5496
Epoch 5/100
29/29 [==============================] - 1s 24ms/step - loss: 0.4573 - accuracy: 0.8540 - val_loss: 0.8135 - val_accuracy: 0.5506
Epoch 6/100
29/29 [==============================] - 0s 15ms/step - loss: 0.4520 - accuracy: 0.8580 - val_loss: 0.8394 - val_accuracy: 0.5345
Epoch 7/100
29/29 [==============================] - 1s 24ms/step - loss: 0.4610 - accuracy: 0.8545 - val_loss: 0.7820 - val_accuracy: 0.6455
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 7s 62ms/step - loss: 0.4635 - accuracy: 0.8466 - val_loss: 0.9425 - val_accuracy: 0.5057
Epoch 2/100
28/28 [==============================] - 1s 32ms/step - loss: 0.4658 - accuracy: 0.8475 - val_loss: 0.8955 - val_accuracy: 0.5068
Epoch 3/100
28/28 [==============================] - 1s 35ms/step - loss: 0.4599 - accuracy: 0.8529 - val_loss: 0.8592 - val_accuracy: 0.5102
Epoch 4/100
28/28 [==============================] - 1s 31ms/step - loss: 0.4497 - accuracy: 0.8554 - val_loss: 0.8505 - val_accuracy: 0.5158
Epoch 5/100
28/28 [==============================] - 1s 35ms/step - loss: 0.4455 - accuracy: 0.8594 - val_loss: 0.8438 - val_accuracy: 0.5249
Epoch 6/100
28/28 [==============================] - 0s 17ms/step - loss: 0.4499 - accuracy: 0.8568 - val_loss: 0.9129 - val_accuracy: 0.5102
Epoch 7/100
28/28 [==============================] - 1s 25ms/step - loss: 0.4557 - accuracy: 0.8526 - val_loss: 0.8186 - val_accuracy: 0.5962
Epoch 8/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 8s 56ms/step - loss: 0.4952 - accuracy: 0.8336 - val_loss: 0.9005 - val_accuracy: 0.5155
Epoch 2/100
31/31 [==============================] - 1s 24ms/step - loss: 0.4762 - accuracy: 0.8452 - val_loss: 0.8447 - val_accuracy: 0.5186
Epoch 3/100
31/31 [==============================] - 0s 15ms/step - loss: 0.4657 - accuracy: 0.8535 - val_loss: 0.8695 - val_accuracy: 0.5186
Epoch 4/100
31/31 [==============================] - 1s 23ms/step - loss: 0.4743 - accuracy: 0.8457 - val_loss: 0.8493 - val_accuracy: 0.5248
Epoch 5/100
31/31 [==============================] - 1s 27ms/step - loss: 0.4684 - accuracy: 0.8481 - val_loss: 0.8083 - val_accuracy: 0.5558
Epoch 6/100
31/31 [==============================] - 1s 19ms/step - loss: 0.4754 - accuracy: 0.8455 - val_loss: 0.8268 - val_accuracy: 0.5403
Epoch 7/100
31/31 [==============================] - 1s 30ms/step - loss: 0.4566 - accuracy: 0.8581 - val_loss: 0.7777 - val_accuracy: 0.6250
Epoch 8/100
31/31 

In [ ]:

metrics_df.round(3)

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.617,0.634,0.554,0.592,0.554,0.680,0.235
1,1,0.626,0.674,0.490,0.567,0.490,0.763,0.253
2,2,0.642,0.634,0.671,0.652,0.671,0.612,0.283
3,0,0.638,0.647,0.608,0.627,0.608,0.668,0.276
4,1,0.654,0.688,0.564,0.620,0.564,0.744,0.308
5,2,0.651,0.667,0.602,0.633,0.602,0.699,0.301
6,0,0.665,0.668,0.655,0.662,0.655,0.675,0.330
7,1,0.699,0.725,0.641,0.681,0.641,0.757,0.398
8,2,0.673,0.692,0.622,0.655,0.622,0.723,0.345
9,0,0.700,0.728,0.638,0.680,0.638,0.762,0.400


In [ ]:
metrics_df.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/TF Domain/CNN_LSTM/Theta_tf_CNN_LSTM.csv', index = False)

# CNN

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    # model.add(Dropout(0.5))
    # model.add(LSTM(256, return_sequences=True))
    # model.add(LSTM(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Theta/CNN/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Theta/CNN/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
global_model_CNN, metrics_df_CNN = federated_learning(Theta_data)

Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 37, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 37, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 19, 256)           0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 19s 97ms/step - loss: 1.9596 - accuracy: 0.5159 - val_loss: 1.9531 - val_accuracy: 0.5506
Epoch 2/100
29/29 [==============================] - 1s 19ms/step - loss: 1.9380 - accuracy: 0.5245 - val_loss: 1.9351 - val_accuracy: 0.5345
Epoch 3/100
29/29 [==============================] - 1s 26ms/step - loss: 1.9184 - accuracy: 0.5423 - val_loss: 1.9173 - val_accuracy: 0.5690
Epoch 4/100
29/29 [==============================] - 1s 17ms/step - loss: 1.8997 - accuracy: 0.5673 - val_loss: 1.8998 - val_accuracy: 0.5647
Epoch 5/100
29/29 [==============================] - 1s 18ms/step - loss: 1.8814 - accuracy: 0.5676 - val_loss: 1.8825 - val_accuracy: 0.5571
Epoch 6/100
29/29 [==============================] - 0s 17ms/step - loss: 1.8632 - accuracy: 0.5749 - val_loss: 1.8656 - val_accuracy: 0.5528
Epoch 7/100
29/29 [==============================] - 1s 18ms/step - loss: 1.8453 - accuracy: 0.5808 - val_loss: 1.8487 - val_accuracy: 0.5528
Epoch 8/100
29/29

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 1s 19ms/step - loss: 1.9394 - accuracy: 0.5164 - val_loss: 1.9361 - val_accuracy: 0.5781
Epoch 3/100
28/28 [==============================] - 0s 13ms/step - loss: 1.9206 - accuracy: 0.5280 - val_loss: 1.9189 - val_accuracy: 0.5475
Epoch 4/100
28/28 [==============================] - 0s 14ms/step - loss: 1.9026 - accuracy: 0.5498 - val_loss: 1.9018 - val_accuracy: 0.5464
Epoch 5/100
28/28 [==============================] - 0s 14ms/step - loss: 1.8847 - accuracy: 0.5608 - val_loss: 1.8850 - val_accuracy: 0.5475
Epoch 6/100
28/28 [==============================] - 0s 15ms/step - loss: 1.8673 - accuracy: 0.5645 - val_loss: 1.8686 - val_accuracy: 0.5362
Epoch 7/100
28/28 [==============================] - 0s 16ms/step - loss: 1.8499 - accuracy: 0.5676 - val_loss: 1.8523 - val_accuracy: 0.5339
Epoch 8/100
28/28 [==============================] - 0s 16ms/step - loss: 1.8327 - accuracy: 0.5665 - val_loss: 1.8362 - val_accuracy: 0.5441
Epoch 9/100
28/28 

...

29/29 [==============================] - 0s 11ms/step - loss: 0.3778 - accuracy: 0.9469 - val_loss: 0.8617 - val_accuracy: 0.6228
Epoch 3/100
29/29 [==============================] - 0s 15ms/step - loss: 0.3643 - accuracy: 0.9485 - val_loss: 0.8824 - val_accuracy: 0.5657
Epoch 4/100
29/29 [==============================] - 0s 14ms/step - loss: 0.3536 - accuracy: 0.9561 - val_loss: 0.8690 - val_accuracy: 0.5905
Epoch 5/100
29/29 [==============================] - 0s 13ms/step - loss: 0.3464 - accuracy: 0.9582 - val_loss: 0.8983 - val_accuracy: 0.5636
Epoch 6/100
29/29 [==============================] - 0s 15ms/step - loss: 0.3510 - accuracy: 0.9542 - val_loss: 0.8497 - val_accuracy: 0.6347
Epoch 7/100
29/29 [==============================] - 1s 24ms/step - loss: 0.3505 - accuracy: 0.9550 - val_loss: 0.8349 - val_accuracy: 0.6606
Epoch 8/100
29/29 [==============================] - 0s 16ms/step - loss: 0.3433 - accuracy: 0.9591 - val_loss: 0.8379 - val_accuracy: 0.6433
Epoch 9/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 1s 19ms/step - loss: 0.3858 - accuracy: 0.9437 - val_loss: 0.8832 - val_accuracy: 0.5814
Epoch 3/100
28/28 [==============================] - 0s 14ms/step - loss: 0.3726 - accuracy: 0.9485 - val_loss: 0.9453 - val_accuracy: 0.5271
Epoch 4/100
28/28 [==============================] - 0s 12ms/step - loss: 0.3695 - accuracy: 0.9465 - val_loss: 0.9018 - val_accuracy: 0.5509
Epoch 5/100
28/28 [==============================] - 0s 14ms/step - loss: 0.3556 - accuracy: 0.9539 - val_loss: 0.9001 - val_accuracy: 0.5543
Epoch 6/100
28/28 [==============================] - 0s 13ms/step - loss: 0.3543 - accuracy: 0.9564 - val_loss: 0.8696 - val_accuracy: 0.5792
Epoch 7/100
28/28 [==============================] - 0s 14ms/step - loss: 0.3556 - accuracy: 0.9561 - val_loss: 0.9001 - val_accuracy: 0.5622
Epoch 8/100
28/28 [==============================] - 0s 12ms/step - loss: 0.3567 - accuracy: 0.9590 - val_loss: 0.9255 - val_accuracy: 0.5543
Epoch 9/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 1s 20ms/step - loss: 0.3973 - accuracy: 0.9326 - val_loss: 0.8870 - val_accuracy: 0.5826
Epoch 3/100
31/31 [==============================] - 0s 14ms/step - loss: 0.3924 - accuracy: 0.9385 - val_loss: 0.8870 - val_accuracy: 0.5682
Epoch 4/100
31/31 [==============================] - 0s 13ms/step - loss: 0.3977 - accuracy: 0.9341 - val_loss: 0.9170 - val_accuracy: 0.5475
Epoch 5/100
31/31 [==============================] - 0s 14ms/step - loss: 0.3874 - accuracy: 0.9364 - val_loss: 0.9236 - val_accuracy: 0.5403
Epoch 6/100
31/31 [==============================] - 0s 14ms/step - loss: 0.3778 - accuracy: 0.9447 - val_loss: 0.9208 - val_accuracy: 0.5568
Epoch 7/100
31/31 [==============================] - 1s 18ms/step - loss: 0.3687 - accuracy: 0.9491 - val_loss: 0.8767 - val_accuracy: 0.5961
Epoch 8/100
31/31 [==============================] - 0s 14ms/step - loss: 0.3712 - accuracy: 0.9488 - val_loss: 0.9097 - val_accuracy: 0.5795
Epoch 9/100
31/31 

In [ ]:
metrics_df_CNN.round(3)

,Client,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,Kappa
0,0,0.599,0.635,0.464,0.536,0.464,0.734,0.198
1,1,0.633,0.696,0.475,0.564,0.475,0.792,0.267
2,2,0.613,0.615,0.606,0.611,0.606,0.620,0.227
3,0,0.643,0.678,0.544,0.604,0.544,0.742,0.286
4,1,0.676,0.694,0.630,0.660,0.630,0.722,0.352
5,2,0.677,0.742,0.542,0.626,0.542,0.811,0.353
6,0,0.689,0.736,0.590,0.655,0.590,0.789,0.379
7,1,0.724,0.763,0.650,0.702,0.650,0.798,0.448
8,2,0.714,0.702,0.743,0.722,0.743,0.685,0.428
9,0,0.729,0.761,0.668,0.712,0.668,0.791,0.459


In [ ]:
metrics_df_CNN.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/TF Domain/CNN/Theta_tf_CNN.csv', index = False)

# GRU

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, LSTM, Dropout, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import tensorflow_addons as tfa

class FederatedData:
    def __init__(self, data_path, num_clients, scaler_type='MinMax', test_size=0.2):
        self.data_path = data_path
        self.num_clients = num_clients
        self.scaler_type = scaler_type
        self.test_size = test_size
        self.partitions = []  # Initialize partitions attribute
        self.load_data()
        self.scale_data()
        self.create_partitions()

    def load_data(self):
        try:
            data = np.load(self.data_path, allow_pickle=True)
            self.X = np.moveaxis(data['X'], 1, 2)  # Move axis here
            self.Y = data['Y']
            self.groups = data['Group']  # Assuming 'Group' contains group identifiers
        except KeyError as e:
            raise ValueError(f"Missing expected data field: {e}")
        except FileNotFoundError as e:
            raise ValueError(f"Data file not found: {e}")

    def scale_data(self):
        # Reshape data to 2D array for scaling
        X_reshaped = self.X.reshape(-1, self.X.shape[-1])

        # Select scaler based on input
        if self.scaler_type == 'Standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'MinMax':
            scaler = MinMaxScaler(feature_range=(0, 1))
        else:
            raise ValueError("Unsupported scaler type. Choose either 'Standard' or 'MinMax'.")

        # Fit and transform the data
        scaled_data_reshaped = scaler.fit_transform(X_reshaped)

        # Reshape back to original shape
        self.X = scaled_data_reshaped.reshape(self.X.shape)

    def create_partitions(self):
        gss = GroupShuffleSplit(n_splits=self.num_clients, test_size=self.test_size, random_state=42)
        for train_index, test_index in gss.split(self.X, self.Y, self.groups):
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.Y[train_index], self.Y[test_index]
            self.partitions.append((X_train, Y_train, X_test, Y_test))
            print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

    def get_training_and_validation_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        partition_X_train, partition_Y_train, partition_X_test, partition_Y_test = self.partitions[client_idx]
        X_train, X_val, Y_train, Y_val = train_test_split(partition_X_train, partition_Y_train, test_size=0.2, random_state=42)
        return X_train, X_val, Y_train, Y_val, partition_X_test, partition_Y_test

    def get_testing_data(self, client_idx):
        if client_idx < 0 or client_idx >= len(self.partitions):
            raise ValueError(f"Invalid client index. Must be between 0 and {len(self.partitions) - 1}.")

        _, _, partition_X_test, partition_Y_test = self.partitions[client_idx]
        return partition_X_test, partition_Y_test

def build_sequential_model_gru(input_shape):
    clear_session()
    model = Sequential()

    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(256, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(512, 3, strides=1, activation='relu', padding="same"))
    model.add(MaxPooling1D(2, padding="same"))
    model.add(Conv1D(filters=1024, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(GRU(256, return_sequences=True))
    model.add(GRU(256, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(32, activation='relu',kernel_regularizer=l2(0.001)))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

    # opt = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=0.00009)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = recall

    return accuracy, precision, recall, f1, sensitivity, specificity, kappa

def federated_learning_gru(data_path):
    federated_data = FederatedData(data_path, num_clients=3, scaler_type='MinMax')
    federated_data.create_partitions()

    # Get the input shape from the data
    input_shape = federated_data.X.shape[1:]
    global_model = build_sequential_model_gru(input_shape)

    num_clients = 3
    local_epochs = 5
    global_optimizer = Adam(learning_rate=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    # Initialize m and v for Adam optimizer
    m = [np.zeros_like(w) for w in global_model.get_weights()]
    v = [np.zeros_like(w) for w in global_model.get_weights()]
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-7
    t = 0

    client_data = []
    for client_idx in range(num_clients):
        x_train, x_val, y_train, y_val, x_test, y_test = federated_data.get_training_and_validation_data(client_idx)
        client_data.append((x_train, x_val, y_train, y_val, x_test, y_test))

    metrics_list = []

    for epoch in range(local_epochs):
        client_models = []

        for client in range(num_clients):
            x_train, x_val, y_train, y_val, x_test, y_test = client_data[client]
            client_model = build_sequential_model_gru(input_shape)
            client_model.set_weights(global_model.get_weights())  # Initialize with global weights
            opt = Adam(learning_rate=0.000009, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            client_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

            # Set up callbacks
            run_name = f"epoch_{epoch}_client_{client}"
            checkpoint_filepath = f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Theta/CNN_GRU/best_model_{run_name}.h5'
            checkpoint_callback = ModelCheckpoint(
                checkpoint_filepath,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            )
            csv_logger = CSVLogger(f'/content/drive/MyDrive/EEG Signal /Epileptic seizure/Saved model/Time+Frequency/Theta/CNN_GRU/training_log_{run_name}.csv', append=True, separator=';')

            history = client_model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), callbacks=[checkpoint_callback, csv_logger])
            client_models.append(client_model)

            print(history.history)

            # Load the best model before evaluation
            best_model = load_model(checkpoint_filepath)

            # Evaluate client model
            y_pred = (best_model.predict(x_test) > 0.5).astype("int32")
            accuracy, precision, recall, f1, sensitivity, specificity, kappa = compute_metrics(y_test, y_pred)
            metrics_list.append({
                'Client': client,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Sensitivity': sensitivity,
                'Specificity': specificity,
                'Kappa': kappa
            })

        global_weights = global_model.get_weights()
        layer_start_idx = 0
        for layer in global_model.layers:
            layer_weights = []
            layer_biases = []
            num_params = len(layer.get_weights())
            for i in range(num_clients):
                layer_params = client_models[i].get_weights()[layer_start_idx:layer_start_idx + num_params]
                if num_params > 0:
                    layer_weights.append(layer_params[0])
                if num_params > 1:
                    layer_biases.append(layer_params[1])

            if len(layer_weights) > 0:
                averaged_layer_weights = np.mean(layer_weights, axis=0)
                if averaged_layer_weights.shape == global_weights[layer_start_idx].shape:
                    global_weights[layer_start_idx] = averaged_layer_weights
                else:
                    print(f"Warning: Shape mismatch for weights at layer {layer.name}, expected {global_weights[layer_start_idx].shape} but got {averaged_layer_weights.shape}")
            if len(layer_biases) > 0:
                averaged_layer_biases = np.mean(layer_biases, axis=0)
                if averaged_layer_biases.shape == global_weights[layer_start_idx + 1].shape:
                    global_weights[layer_start_idx + 1] = averaged_layer_biases
                else:
                    print(f"Warning: Shape mismatch for biases at layer {layer.name}, expected {global_weights[layer_start_idx + 1].shape} but got {averaged_layer_biases.shape}")

            layer_start_idx += num_params

        # Apply FedOpt (Adam) update to global weights
        t += 1
        for i in range(len(global_weights)):
            g_t = global_weights[i] - global_model.get_weights()[i]  # Gradient
            m[i] = beta1 * m[i] + (1 - beta1) * g_t
            v[i] = beta2 * v[i] + (1 - beta2) * (g_t ** 2)
            m_hat = m[i] / (1 - beta1 ** t)
            v_hat = v[i] / (1 - beta2 ** t)
            global_weights[i] -= global_optimizer.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        global_model.set_weights(global_weights)

    metrics_df = pd.DataFrame(metrics_list)
    return global_model, metrics_df

# Example usage
# data_path = 'path/to/your/data.npz'
# global_model, metrics_df = federated_learning(data_path)
# print(metrics_df)



In [ ]:
global_model_gru, metrics_df_gru = federated_learning_gru(Theta_data)

Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Train shape: (4640, 37, 29), Test shape: (1194, 37, 29)
Train shape: (4418, 37, 29), Test shape: (1416, 37, 29)
Train shape: (4838, 37, 29), Test shape: (996, 37, 29)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 37, 256)           22528     
                                                                 
 batch_normalization (Batch  (None, 37, 256)           1024      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 19, 256)           0         
 D)                                                              
                                                    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


29/29 [==============================] - 8s 72ms/step - loss: 1.7018 - accuracy: 0.5374 - val_loss: 1.6971 - val_accuracy: 0.5647
Epoch 2/100
29/29 [==============================] - 0s 17ms/step - loss: 1.6906 - accuracy: 0.5458 - val_loss: 1.6871 - val_accuracy: 0.5474
Epoch 3/100
29/29 [==============================] - 1s 22ms/step - loss: 1.6793 - accuracy: 0.5620 - val_loss: 1.6772 - val_accuracy: 0.5442
Epoch 4/100
29/29 [==============================] - 1s 19ms/step - loss: 1.6679 - accuracy: 0.5690 - val_loss: 1.6674 - val_accuracy: 0.5420
Epoch 5/100
29/29 [==============================] - 1s 18ms/step - loss: 1.6567 - accuracy: 0.5657 - val_loss: 1.6575 - val_accuracy: 0.5539
Epoch 6/100
29/29 [==============================] - 1s 18ms/step - loss: 1.6457 - accuracy: 0.5700 - val_loss: 1.6478 - val_accuracy: 0.5560
Epoch 7/100
29/29 [==============================] - 0s 15ms/step - loss: 1.6351 - accuracy: 0.5760 - val_loss: 1.6382 - val_accuracy: 0.5550
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 0s 16ms/step - loss: 1.6915 - accuracy: 0.5450 - val_loss: 1.6878 - val_accuracy: 0.5351
Epoch 3/100
28/28 [==============================] - 0s 14ms/step - loss: 1.6808 - accuracy: 0.5410 - val_loss: 1.6782 - val_accuracy: 0.5362
Epoch 4/100
28/28 [==============================] - 0s 15ms/step - loss: 1.6698 - accuracy: 0.5557 - val_loss: 1.6688 - val_accuracy: 0.5305
Epoch 5/100
28/28 [==============================] - 0s 15ms/step - loss: 1.6590 - accuracy: 0.5591 - val_loss: 1.6594 - val_accuracy: 0.5305
Epoch 6/100
28/28 [==============================] - 0s 16ms/step - loss: 1.6486 - accuracy: 0.5600 - val_loss: 1.6500 - val_accuracy: 0.5396
Epoch 7/100
28/28 [==============================] - 0s 15ms/step - loss: 1.6383 - accuracy: 0.5625 - val_loss: 1.6408 - val_accuracy: 0.5385
Epoch 8/100
28/28 [==============================] - 0s 15ms/step - loss: 1.6286 - accuracy: 0.5665 - val_loss: 1.6314 - val_accuracy: 0.5419
Epoch 9/100
28/28 

...

29/29 [==============================] - 7s 68ms/step - loss: 0.4284 - accuracy: 0.8869 - val_loss: 0.8375 - val_accuracy: 0.5625
Epoch 2/100
29/29 [==============================] - 1s 27ms/step - loss: 0.4178 - accuracy: 0.8955 - val_loss: 0.8343 - val_accuracy: 0.5797
Epoch 3/100
29/29 [==============================] - 1s 32ms/step - loss: 0.4172 - accuracy: 0.8860 - val_loss: 0.8241 - val_accuracy: 0.6315
Epoch 4/100
29/29 [==============================] - 1s 17ms/step - loss: 0.4045 - accuracy: 0.8930 - val_loss: 0.8276 - val_accuracy: 0.5905
Epoch 5/100
29/29 [==============================] - 1s 18ms/step - loss: 0.3890 - accuracy: 0.9054 - val_loss: 0.8330 - val_accuracy: 0.5841
Epoch 6/100
29/29 [==============================] - 1s 22ms/step - loss: 0.3916 - accuracy: 0.9038 - val_loss: 0.8141 - val_accuracy: 0.6336
Epoch 7/100
29/29 [==============================] - 0s 15ms/step - loss: 0.3929 - accuracy: 0.9009 - val_loss: 0.8250 - val_accuracy: 0.6024
Epoch 8/100
29/29 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


28/28 [==============================] - 8s 56ms/step - loss: 0.4416 - accuracy: 0.8749 - val_loss: 0.8831 - val_accuracy: 0.5170
Epoch 2/100
28/28 [==============================] - 1s 24ms/step - loss: 0.4269 - accuracy: 0.8809 - val_loss: 0.8245 - val_accuracy: 0.6233
Epoch 3/100
28/28 [==============================] - 0s 16ms/step - loss: 0.4308 - accuracy: 0.8860 - val_loss: 0.8277 - val_accuracy: 0.6063
Epoch 4/100
28/28 [==============================] - 1s 23ms/step - loss: 0.4132 - accuracy: 0.8899 - val_loss: 0.8255 - val_accuracy: 0.6335
Epoch 5/100
28/28 [==============================] - 0s 17ms/step - loss: 0.4202 - accuracy: 0.8857 - val_loss: 0.8120 - val_accuracy: 0.5984
Epoch 6/100
28/28 [==============================] - 0s 16ms/step - loss: 0.4061 - accuracy: 0.8970 - val_loss: 0.8044 - val_accuracy: 0.6324
Epoch 7/100
28/28 [==============================] - 0s 16ms/step - loss: 0.4093 - accuracy: 0.8916 - val_loss: 0.8560 - val_accuracy: 0.5566
Epoch 8/100
28/28 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


31/31 [==============================] - 11s 61ms/step - loss: 0.4427 - accuracy: 0.8755 - val_loss: 0.8237 - val_accuracy: 0.6054
Epoch 2/100
31/31 [==============================] - 1s 16ms/step - loss: 0.4530 - accuracy: 0.8721 - val_loss: 0.8326 - val_accuracy: 0.5981
Epoch 3/100
31/31 [==============================] - 0s 15ms/step - loss: 0.4308 - accuracy: 0.8835 - val_loss: 0.8330 - val_accuracy: 0.5579
Epoch 4/100
31/31 [==============================] - 0s 16ms/step - loss: 0.4193 - accuracy: 0.8899 - val_loss: 0.8262 - val_accuracy: 0.5775
Epoch 5/100
31/31 [==============================] - 1s 17ms/step - loss: 0.4094 - accuracy: 0.8969 - val_loss: 0.8147 - val_accuracy: 0.6023
Epoch 6/100
31/31 [==============================] - 0s 16ms/step - loss: 0.4070 - accuracy: 0.8933 - val_loss: 0.8100 - val_accuracy: 0.5971
Epoch 7/100
31/31 [==============================] - 1s 24ms/step - loss: 0.4117 - accuracy: 0.8922 - val_loss: 0.7894 - val_accuracy: 0.6539
Epoch 8/100
31/31

In [ ]:
metrics_df_gru

In [ ]:
metrics_df_gru.round(4).to_csv('/content/drive/MyDrive/EEG Signal /results/FLresults/TF Domain/GRU/Theta_tf_GRU.csv', index = False)

In [ ]:
print('hello')

hello
